In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time


chrome_path = r'C:\Users\Girish\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe' 

options = Options()
options.add_argument("--headless")  
driver_service = Service(chrome_path)


driver = webdriver.Chrome(service=driver_service, options=options)


url = "https://www.cricbuzz.com/cricket-scorecard-archives"
driver.get(url)


time.sleep(3)


page_source = driver.page_source
print(page_source) 


driver.quit()


<html lang="en" itemscope="" itemtype="http://schema.org/WebPage"><head><style>@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style><meta charset="utf-8"><script type="text/javascript" async="" src="https://sb.scorecardresearch.com/c2/6036484/cs.js"></script><script type="text/javascript" async="" src="https://d2r1yp2w7bby2u.cloudfront.net/js/clevertap.min.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-4H06J8XXQH&amp;l=dataLayer&amp;cx=c&amp;gtm=45He54g3v71840711za200&amp;tag_exp=102803279~102813109~102887800~102926062~103027016~103051953~103055465~103077950~103106314~103106316"></script><script async="" type="text/javascript" src="https://www.googletagservices.com/tag/js/gpt.js"></script><script type="text/javascript" async="" src="//ade.cl

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

chrome_path = r"C:\chromedriver.exe"

# Set up Selenium
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(chrome_path), options=options)

# Target URL
url = "https://www.cricbuzz.com/cricket-scorecard-archives"
driver.get(url)
time.sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

series_data = []

# Find each year section
years = soup.select(".cb-schdl-hdr, .cb-arcv-yr")
series_blocks = soup.select(".cb-srs-cat, .cb-srs-lst-itm")

current_year = None
current_type = None

for block in series_blocks:
    if "cb-arcv-yr" in block.get("class", []):
        current_year = block.text.strip()
    elif "cb-srs-cat" in block.get("class", []):
        current_type = block.text.strip()
    elif "cb-srs-lst-itm" in block.get("class", []):
        anchor = block.find("a")
        if anchor:
            series_name = anchor.text.strip()
            series_href = "https://www.cricbuzz.com" + anchor.get("href", "")
            series_data.append({
                "year": current_year,
                "series_type": current_type,
                "series_name": series_name,
                "series_href": series_href
            })

df = pd.DataFrame(series_data)
print(df.head())
df.to_csv("cricbuzz_series_data.csv", index=False)


   year    series_type                                   series_name  \
0  None  International       Malaysia Quadrangular T20I Series, 2025   
1  None  International             Zimbabwe tour of Bangladesh, 2025   
2  None  International              The North American T20 Cup, 2025   
3  None  International  Central American Cricket Championships, 2025   
4  None  International      Hong Kong T20I Quadrangular Series, 2025   

                                         series_href  
0  https://www.cricbuzz.com/cricket-series/9710/m...  
1  https://www.cricbuzz.com/cricket-series/9514/z...  
2  https://www.cricbuzz.com/cricket-series/9677/t...  
3  https://www.cricbuzz.com/cricket-series/9692/c...  
4  https://www.cricbuzz.com/cricket-series/9647/h...  


In [4]:
df = pd.read_csv('cricbuzz_series_data.csv')
df.columns

Index(['year', 'series_type', 'series_name', 'series_href'], dtype='object')

In [5]:
df['series_name'] = df['series_name'].astype(str)
df['year'] = df['series_name'].str.extract(r',\s*(\d{4})')
df['series_name'] = df['series_name'].str.replace(r',\s*\d{4}', '', regex=True)

df.to_csv('cricbuzz_series_data.csv', index=False)